<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_gpt_TR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!pip install sacrebleu
!pip install evaluate
!pip install rouge_score
!pip install bert_score

In [86]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import openai
import pandas as pd
import numpy as np
import os
import kagglehub
from bert_score import score


from google.colab import userdata

# print(openai.__version__)
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_NLP')

In [87]:

import sacrebleu
from nltk.translate.meteor_score import meteor_score
import nltk
import evaluate

# Download necessary resources for METEOR
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but recommended
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [134]:
# import data from google drive
file_name = "data_eng_greek_1000"
# 'data_eng_greek_1000'
# "data_eng_greek_200"

path = "/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/"+file_name+".csv"
data = pd.read_csv(path)
data.head()



,english,greek
0,Tom works.,Ο Τομ δουλεύει.
1,This is a family tradition.,Αυτή είναι μια οικογενειακή παράδοση.
2,Tom admired Mary.,Ο Τομ θαύμαζε τη Μαίρη.
3,I was all ears.,Ήμουν όλος αυτιά.
4,Give me that bottle.,Δώσε μου εκείνο το μπουκάλι.


In [135]:
len(data)

1000

In [136]:
zero_shot = f""" """

one_shot = f"""
              Input: "I would ike to eat something."
              Output: "θα ήθελα να φάω κάτι."
"""

three_shot = f"""
              Input: "I understand a little greek."
              Output: "καταλαβαίνω λίγα ελληνικά."
              Input: "Where is the bathroom?"
              Output: "πού είναι το τουαλέτα?"
              Input: "My grandma is greek."
              Output: "η γιαγιά μου είναι Ελληνίδα."
"""


In [137]:
# text classification with GPT

def translate_text(text):
    try:
        # Call GPT-3.5 Turbo
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a translator assistant who translates from English to Greek. Given a phrase in English, repsond with the translated phrase in Greek."},
                {"role": "assistant", "content": shot},
                {"role": "user", "content": text}
            ]
        )
        # Extract and return the response
        translation = response.choices[0].message.content.strip()
#       print(response.choices[0].message.content)
        return translation
    except Exception as e:
        return f"Error: {e}"

In [138]:
# adjust for desired prompt / model / data size
test_size = len(data)   # can adjust for smaller testing

function = translate_text
shot_name = "three_shot"      # change for zero / one / three shot
model_name = "gpt-3.5-turbo"  # change for different models

In [139]:

prediction = []

if shot_name == "one_shot":
  shot = one_shot
elif shot_name == "three_shot":
  shot = three_shot
else:
  shot = zero_shot



def translate(data):
  count = 0
  for i, texts in enumerate(data.english[:test_size]):
    result = function(texts)
    count += 1
    if count % 100 == 0:
      print(count)

    prediction.append(result)



In [140]:
translate(data)


100
200
300
400
500
600
700
800
900
1000


In [141]:
len(prediction)

1000

In [142]:
# greek to list
reference = data.greek[:test_size].tolist()


In [143]:
file_name

'data_eng_greek_1000'

In [144]:
shot_name


'three_shot'

In [129]:
'''# print predictions and references to compare
for i in range(len(prediction)):
  print(f'{prediction[i]} \n{reference[i]}\n')
'''

"# print predictions and references to compare\nfor i in range(len(prediction)):\n  print(f'{prediction[i]} \n{reference[i]}\n')\n"

In [145]:
size = len(prediction)


In [146]:
# view metrics and save
def view_and_save_metrics(prediction, reference):
  # BLEU
  bleu = sacrebleu.corpus_bleu(prediction, [reference])
#  print(f"BLEU: {bleu.score}")

  # Tokenize sentences for METEOR
  prediction_tokens = [nltk.word_tokenize(pred) for pred in prediction]
  reference_tokens = [[nltk.word_tokenize(ref)] for ref in reference]  # Note the double list for reference

  # METEOR (sentence-level average)
  meteor_scores = [
      meteor_score(ref_tokens, pred_tokens)
      for ref_tokens, pred_tokens in zip(reference_tokens, prediction_tokens)
  ]
  avg_meteor = sum(meteor_scores) / len(meteor_scores)

  # ROUGE
  rouge = evaluate.load('rouge')
  rouge_scores = rouge.compute(predictions=prediction, references=reference)
#  print(f"ROUGE: {rouge_scores}")

  # BERT Score
  P_, R_, F1 = score(prediction, reference, lang="el")
  # print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
  bert_score = F1.mean().item()

  metrics = {
      "bleu": bleu.score,
      "meteor": avg_meteor,
      "bert_score": bert_score,
      "model_name": model_name,
      "dataset": file_name,
      "shot": shot_name,
      "size": size
  }

  metrics.update(rouge_scores)
  '''
  # Print all metrics
  print("\n--- Evaluation Metrics ---")
  print(f"BLEU:   {bleu.score:.4f}")
  print(f"METEOR: {avg_meteor:.4f}")
  for k, v in rouge_scores.items():
      print(f"{k.upper()}: {v:.4f}")
  '''

  # save metrics to csv file
  metrics_df = pd.DataFrame([metrics])
  metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/metrics/translation/metrics_'+model_name+'_'+file_name+'_'+shot_name+'_'+str(size)+'.csv', index=False)

  return metrics

In [147]:
metrics = view_and_save_metrics(prediction, reference)
#metrics = view_and_save_metrics(cleaned_predictions, cleaned_references)

In [148]:
metrics

{'bleu': 46.674290126721225,
 'meteor': 0.7081326151268252,
 'bert_score': 0.9215567708015442,
 'model_name': 'gpt-3.5-turbo',
 'dataset': 'data_eng_greek_1000',
 'shot': 'three_shot',
 'size': 1000,
 'rouge1': 0.0078,
 'rouge2': 0.0055,
 'rougeL': 0.0078,
 'rougeLsum': 0.0078}